# Data Analysis: Restaurants in Mumbai & Thane

### This is the code for the final capstone project in the IBM Data Science Course.  
If you feel like you have any questions about my work, feel free to contact me at pronoy.55@gmail.com  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
import json
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup as bs
import time

The code below is used to get the list of all the URLs that contain the pincodes since it is seperated by areas within Maharashtra. During this step I realized that the entire state would be too big of a project to work with and complete within a week. Hence, I did make the entire URL list but ended up using just 'list16' which is the list of pincodes in Mumbai and Suburban Mumbai. The pincodes for thane were not giving reliable results and so I ended up using a different website to scrap the data for area localities in thane.

In [2]:
url = requests.get('https://finkode.com/maharashtra.html').text #This website has a list of pincodes for the state of Maharashtra
soup = bs(url)
table = soup.find('table')
setT = table.findAll('td')
newlist = []
for k in setT:
    newlist.append(k.text)
    
c=0
newlistnew = []
while newlist:
    newlistnew.append(newlist[c])
    if c == len(newlist)-3:
        break
    c = c+3
    
newlistnew
newlist = newlistnew
newlist[0] = 'Ahmed-nagar' #although I do not use the pincodes for almost all the places in this list, I try and keep-
#-the data clean and therefore getting rid of the spacebar and adding a hyphen helps in keeping the URL porpper.
newlist = [x.strip(' ') for x in newlist]#Gets rid of the spacebar at the end of each name of place to get the porpper URL.

urlList = []
#https://finkode.com/mh/ahmed-nagar.html ##This is a format of the URL in the URL list.
c=0
while newlist:
    urlList.append('https://finkode.com/mh/{}.html'.format(newlist[c]))
    if c==33:
        break
    c=c+1

In [6]:
list16 = [] ######
print(urlList[16])######
insoup = bs(requests.get(urlList[16]).text)######
intable = insoup.find('table')
insetT = intable.findAll('td')
inlist = []
for k in insetT:
    inlist.append(k.text)
c = 2
while inlist:
    list16.append(inlist[c])######
    if c == len(inlist) - 1:
        break
    c = c+3
    
list16[:6]

https://finkode.com/mh/Mumbai.html


['400029', '400065', '400011', '400004', '400069', '400053']

I create an empty Data Frame to append to later:

In [7]:
mumbai_data = pd.DataFrame(columns = ['Pincode', 'Latitude', 'Longitude'])
mumbai_data

,Pincode,Latitude,Longitude


In [8]:
accessToken = 'pk.2014c34a04d76be00c36d67b51d6f0b4'

In [11]:
mumbailat = []
mumbailon = []
missinglist = []
c = 0
while list16:
    if c == len(list16):
        break
    file = requests.get('https://us1.locationiq.com/v1/search.php?key={}&q={} mumbai&viewbox=82.6043347,23.0309689,72.5378861,15.8922222&bounded=1'.format(accessToken, list16[c])).text
    trysoup = bs(file, 'lxml')
    mumbailist = trysoup.findAll('place')
    if mumbailist == []:
        print('Add this', c)
        missinglist.append(c)
        c = c+1
        continue
    mumbailist = mumbailist[0]
    mumbai = str(mumbailist)
    mumbailat.append(mumbai[mumbai.find('lat')+5: mumbai.find('lat')+14])
    mumbailon.append(mumbai[mumbai.find('lon')+5: mumbai.find('lon')+14])
    c=c+1
    time.sleep(0.5) #This helps create a gap between the initial call and the next call so that the program can function smoothly.

print('mumbai_lat&lon done!')

Add this 0
Add this 17
Add this 85
Add this 191
Add this 227
mumbai_lat&lon done!


The 'Add this: {number}' is basically the index of the pincode that did not retrieve any results and in the next two cells I get rid of the pincodes from the list so that the pincodes and their coordinates are propperly alligned. 

In [12]:
print(list16[0])
print(list16[17])
print(list16[85])
print(list16[191])
print(list16[227])

400029
400085
400056
400029
400056


In [13]:
list16.remove('400029')
list16.remove('400085')
list16.remove('400056')
list16.remove('400029')
list16.remove('400056')

In [14]:
diction = {'Pincode': list16, 'Latitude': mumbailat, 'Longitude': mumbailon}
mumbai_data = pd.DataFrame(diction, columns = ['Pincode','Latitude','Longitude'])

In [15]:
mumbai_data.drop_duplicates(subset = 'Pincode', keep = 'first', inplace = True) 
mumbai_data.reset_index(drop = True)
mumbai_data.drop_duplicates(subset = 'Latitude', keep = 'first', inplace = True)
mumbai_data.reset_index(drop=True, inplace = True)
mumbai_data.head()

,Pincode,Latitude,Longitude
0,400065,19.166233,72.874995
1,400011,18.938771,72.835335
2,400069,19.119689,72.851436
3,400053,19.128015,72.832513
4,400058,19.125732,72.842674


In [16]:
#Zomato Details put below
apiKey = '2b15c7768768f38bbbf45219d7ff1776'

The next cell has a function that makes the process of making a Data Frame with the details from the <b> Zomato API </b> quick and streamlined. It does not remove duplicates and so that step is done after the initial data frame is made.

In [20]:
def newFrame(lat, lon):
    geocode = requests.get('https://developers.zomato.com/api/v2.1/search?count=50&lat={}&lon={}&sort=real_distance&order=asc'.format(lat, lon), headers = {'user-key':'2b15c7768768f38bbbf45219d7ff1776', 'Accept': 'application/json'})
    data = geocode.json()
    name, avg, local, latitude, longitude, cuisines = [],[],[],[],[],[]
    c = 0
    while True:
        if c == len(data['restaurants']):
            break
        
        diction = data['restaurants'][c]
        name.append(diction['restaurant']['name'])
        avg.append(diction['restaurant']['average_cost_for_two'])
        local.append(diction['restaurant']['location']['locality'])
        latitude.append(diction['restaurant']['location']['latitude'])
        longitude.append(diction['restaurant']['location']['longitude'])
        cuisines.append(diction['restaurant']['cuisines'])
        c =c+1
        
    diction = {'Name': name, 'Average Cost for Two': avg, 'Locality':local,'Latitude':latitude, 'Longitude':longitude, 'Cuisines':cuisines}
    FrameName = pd.DataFrame(diction, columns = ['Name','Average Cost for Two', 'Locality', 'Latitude','Longitude','Cuisines'])
    return FrameName

In [21]:
initFrame = pd.DataFrame(columns = ['Name','Average Cost for Two', 'Locality', 'Latitude','Longitude','Cuisines'])
initFrame

,Name,Average Cost for Two,Locality,Latitude,Longitude,Cuisines


In [22]:
c = 0
while True:
    if c == len(mumbai_data['Latitude']):
        break
    finFrame = newFrame(mumbai_data['Latitude'][c], mumbai_data['Longitude'][c])
    initFrame = pd.concat([initFrame, finFrame])
    c = c+1
    
initFrame.head()

,Name,Average Cost for Two,Locality,Latitude,Longitude,Cuisines
0,Asiatic Kouzina,500,Goregaon East,19.1676448343,72.8766452149,"South Indian, North Indian, Chinese, Thai, Mug..."
1,Aashirwad,600,Goregaon East,19.1685245942,72.8756789491,"Chinese, Mughlai, North Indian"
2,My Kitchen Hub,650,Goregaon East,19.1696421801,72.8752236441,"Chinese, North Indian, Mughlai, Kebab, Biryani"
3,Delicious,300,Goregaon East,19.1699417641,72.8749138489,"Italian, Chinese"
4,Chinese Lijiang,300,Goregaon East,19.1700950000,72.8763950000,Chinese


In [23]:
latitude = 19.0760
longitude = 72.8777

In [24]:
MumbaiMap = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, label in zip(initFrame['Latitude'], initFrame['Longitude'], initFrame['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(MumbaiMap)
MumbaiMap

After mapping the data onto a map, there is an apparent blank area in certain parts of mumbai and thane. Since the <b> Zomato API </b> provides the place details in multiple areas, I constructed a list of places that are missing the restaurants. Since I am a local resident of Mumbai, the expertise used to create the list was mine with the help of the map above and what area seem to be either blank or have very few places.

In [45]:
localityChosen = ['Thane West', 'Borivali West', 'Andheri West', 'Goregaon East', 'Juhu', 'Lower Parel', 'Chembur', 'Bhandup','Bandra West', 'Mulund West','Bandra Kurla Complex','Kandivali West', 'Mumbra','Powai','Vile Parle','Mahakali','Khar','Chowpatty','Ghatkopar East','Mira Road','Sion','Kandivali East', 'Mazgaon','Dadar West','Fort', 'Byculla','Kurla','Ghatkopar West','Churchgate','Veera Desai Area','Vikhroli','Malad West','Worli','Marol','Andheri East','Dahisar East','Azad Nagar', 'Mahim','Dahisar West','Goregaon West','Bandra East','Matunga','Charni Road','Jogeshwari','Kalbadevi','Girgaon','Khadak Street','Dadar East','Borivali East','Malad East', 'Mulund East']
# This is a list of places that did not have enough restaurants representing it.

In [46]:
locallat = []
locallon = []
locallist = []
c = 0
while localityChosen:
    if c == len(localityChosen):
        break
    file = requests.get('https://us1.locationiq.com/v1/search.php?key={}&q={}&viewbox=82.6043347,23.0309689,72.5378861,15.8922222&bounded=1'.format(accessToken, localityChosen[c])).text
    trysoup = bs(file, 'lxml')
    local = trysoup.findAll('place')
    if local == []:
        print('Add this', c)
        locallist.append(c)
        c = c+1
        continue
    local = local[0]
    local = str(local)
    locallat.append(local[local.find('lat')+5: local.find('lat')+14])
    locallon.append(local[local.find('lon')+5: local.find('lon')+14])
    c=c+1
    time.sleep(0.5)

print('local_lat&lon done!')

local_lat&lon done!


In [47]:
diction = {'Locality': localityChosen, 'Latitude':locallat,'Longitude':locallon}
areaFrame = pd.DataFrame(diction, columns = ['Locality','Latitude','Longitude'])
areaFrame.head()

,Locality,Latitude,Longitude
0,Thane West,19.191460,72.977564
1,Borivali West,19.229456,72.847990
2,Andheri West,19.117249,72.833968
3,Goregaon East,19.169262,72.855254
4,Juhu,19.107021,72.827527


In [50]:
c = 0
initFrame = pd.DataFrame(columns = ['Name','Average Cost for Two', 'Locality', 'Latitude','Longitude','Cuisines'])
while True:
    if c == len(areaFrame['Latitude']):
        break
    finFrame = newFrame(areaFrame['Latitude'][c], areaFrame['Longitude'][c])
    initFrame = pd.concat([initFrame, finFrame])
    c = c+1
    
initFrame.head()

,Name,Average Cost for Two,Locality,Latitude,Longitude,Cuisines
0,Mamledar Misal,200,"Naupada, Thane West",19.1914736000,72.9771694000,"Maharashtrian, Street Food, Fast Food"
1,Egg-O,200,"Naupada, Thane West",19.1918510000,72.9779650000,Fast Food
2,Hotel Amantran,400,"Naupada, Thane West",19.1910626431,72.9767241329,"North Indian, South Indian, Fast Food"
3,Desi Zaikaa,600,"Khopat, Thane West",19.1907722803,72.9769336805,"North Indian, Chinese, Biryani"
4,Hotel Heera Panna,600,"Naupada, Thane West",19.1905104150,72.9771174118,"Chinese, North Indian, Mughlai, Seafood"


I call testSet = initFrame so that my initial frame is not lost to the data cleaning process. It help me make good comparisons and give an idea of how much data was cleaned.

In [52]:
testSet = initFrame
testSet = testSet['Cuisines'].str.get_dummies(sep = ', ')
print(initFrame.describe())
testSet.head()

            Name  Average Cost for Two     Locality       Latitude  \
count       1020                  1020         1020           1020   
unique       920                    40           83            980   
top     Monginis                   200  Mulund East  19.2494051000   
freq          12                   143           39              8   

            Longitude Cuisines  
count            1020     1020  
unique            978      429  
top     72.8508644000  Chinese  
freq                8       49  


,American,Asian,Awadhi,BBQ,Bakery,Bar Food,Belgian,Beverages,Biryani,British,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
workingSet = pd.concat([initFrame, testSet], axis=1)
workingSet.drop(['Cuisines'], axis = 1, inplace = True)
workingSet.head()

,Name,Average Cost for Two,Locality,Latitude,Longitude,American,Asian,Awadhi,BBQ,Bakery,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,Mamledar Misal,200,"Naupada, Thane West",19.1914736000,72.9771694000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Egg-O,200,"Naupada, Thane West",19.1918510000,72.9779650000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hotel Amantran,400,"Naupada, Thane West",19.1910626431,72.9767241329,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Desi Zaikaa,600,"Khopat, Thane West",19.1907722803,72.9769336805,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hotel Heera Panna,600,"Naupada, Thane West",19.1905104150,72.9771174118,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
workingSet.drop_duplicates(subset = 'Name', keep = 'first', inplace = True)
workingSet.reset_index(drop=True)

,Name,Average Cost for Two,Locality,Latitude,Longitude,American,Asian,Awadhi,BBQ,Bakery,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,Mamledar Misal,200,"Naupada, Thane West",19.1914736000,72.9771694000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Egg-O,200,"Naupada, Thane West",19.1918510000,72.9779650000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hotel Amantran,400,"Naupada, Thane West",19.1910626431,72.9767241329,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Desi Zaikaa,600,"Khopat, Thane West",19.1907722803,72.9769336805,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hotel Heera Panna,600,"Naupada, Thane West",19.1905104150,72.9771174118,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,Food Corner Sai Leela,200,Mulund East,19.1722757135,72.9621771723,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
916,Sahani Tandoori Corner,400,Mulund East,19.1712483998,72.9589085653,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
917,Firangi Flavours,600,Mulund East,19.1690601117,72.9592780396,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
918,Baker's House,200,Mulund East,19.1692222550,72.9590956494,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [102]:
testSet = workingSet

In [103]:
df = workingSet
df.reset_index(drop = True)
df['text_new'] = df['Locality'].str.split(', ').str[1]
df.reset_index(drop = True)
df['text_new'].head()
val = df['text_new']

In [104]:
testSet.drop(columns = ['text_new'], axis = 1, inplace = True)
testSet.reset_index(drop = True)
testSet.head()

,Name,Average Cost for Two,Locality,Latitude,Longitude,American,Asian,Awadhi,BBQ,Bakery,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,Mamledar Misal,200,Thane West,19.1914736000,72.9771694000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Egg-O,200,Thane West,19.1918510000,72.9779650000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hotel Amantran,400,Thane West,19.1910626431,72.9767241329,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Desi Zaikaa,600,Thane West,19.1907722803,72.9769336805,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hotel Heera Panna,600,Thane West,19.1905104150,72.9771174118,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
testSet.insert(3, 'text_new', val)

testSet.reset_index(drop=True, inplace = True)

In [106]:
c = 0
while True:
    if c == len(testSet['Locality']):
        break
    if pd.isnull(testSet['text_new'][c]) == False:
        testSet.replace(testSet['Locality'][c], testSet['text_new'][c], inplace = True)
        c = c+1
    else:
        c = c+1    
        
testSet.head()

,Name,Average Cost for Two,Locality,text_new,Latitude,Longitude,American,Asian,Awadhi,BBQ,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,Mamledar Misal,200,Thane West,NaN,19.1914736000,72.9771694000,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Egg-O,200,Thane West,NaN,19.1918510000,72.9779650000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hotel Amantran,400,Thane West,NaN,19.1910626431,72.9767241329,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Desi Zaikaa,600,Thane West,NaN,19.1907722803,72.9769336805,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hotel Heera Panna,600,Thane West,NaN,19.1905104150,72.9771174118,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
pd.options.display.max_rows = 500

In [108]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Borivali West            30
Mulund East              30
Ghatkopar West           30
Churchgate               26
Mira Road                22
Vile Parle               22
Bandra East              21
Bhandup                  20
Chandrapur Locality      20
Hinganghat Locality      20
Byculla                  20
Thane West               20
Mumbra                   20
Goregaon East            19
Kandivali West           19
Malad East               19
Alibaug                  18
Dahisar                  18
Chembur                  18
Mulund West              18
Girgaum                  18
Fort                     18
Lower Parel              18
Borivali East            18
Matunga East             18
Ghatkopar East           18
Palghar                  18
Bandra Kurla Complex     17
Powai                    17
Khar                     17
Kandivali East           17
Bandra West              17
Worli                    17
Marol               

<b> Please Note: </b> I have taken some of the area names provided by <b> Zomato API </b> and renamed them such that they are grouped with their closest area available in the list if the number of restaurants was less than 7. Since the list was small, this process was done by hand and liberty was taken to assign areas.

In [109]:
testSet.loc[(testSet.Locality == 'Sakinaka'),'Locality']='Vile Parle'
testSet.loc[(testSet.Locality == 'Vile Parle East'),'Locality']='Vile Parle'
testSet.loc[(testSet.Locality == 'Vile Parle West'),'Locality']='Vile Parle'
testSet.loc[(testSet.Locality == 'Charni Road'),'Locality']='Girgaum'
testSet.loc[(testSet.Locality == 'Santacruz West'),'Locality']='Khar'
testSet.loc[(testSet.Locality == 'Dahisar East'),'Locality']='Dahisar'
testSet.loc[(testSet.Locality == 'Dahisar West'),'Locality']='Dahisar'
testSet.loc[(testSet.Locality == 'Prabhadevi'),'Locality']='Dadar'
testSet.loc[(testSet.Locality == 'Tardeo'),'Locality']='Grant Road'
testSet.loc[(testSet.Locality == 'Bhayandar'),'Locality']='Mira Road'
testSet.loc[(testSet.Locality == 'Dadar East'),'Locality']='Dadar'
testSet.loc[(testSet.Locality == 'Dadar West'),'Locality']='Dadar'

In [110]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Borivali West            30
Mulund East              30
Ghatkopar West           30
Churchgate               26
Mira Road                22
Vile Parle               22
Bandra East              21
Bhandup                  20
Chandrapur Locality      20
Hinganghat Locality      20
Byculla                  20
Thane West               20
Mumbra                   20
Goregaon East            19
Kandivali West           19
Malad East               19
Alibaug                  18
Dahisar                  18
Chembur                  18
Mulund West              18
Girgaum                  18
Fort                     18
Lower Parel              18
Borivali East            18
Matunga East             18
Ghatkopar East           18
Palghar                  18
Bandra Kurla Complex     17
Powai                    17
Khar                     17
Kandivali East           17
Bandra West              17
Worli                    17
Marol               

In [111]:
testSet.loc[(testSet.Locality == 'Kamla Mills Compound'),'Locality']='Lower Parel'
testSet.loc[(testSet.Locality == 'Parel'),'Locality']='Lower Parel'
testSet.loc[(testSet.Locality == 'Mumbai CST Area'),'Locality']='Churchgate'
testSet.loc[(testSet.Locality == 'Marine Lines'),'Locality']='Churchgate'
testSet.loc[(testSet.Locality == 'Near Andheri West Station'),'Locality']='Andheri West'
testSet.loc[(testSet.Locality == 'Near Andheri East Station'),'Locality']='Andhehri East'

In [112]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Borivali West            30
Mulund East              30
Ghatkopar West           30
Churchgate               26
Mira Road                22
Vile Parle               22
Bandra East              21
Bhandup                  20
Chandrapur Locality      20
Hinganghat Locality      20
Byculla                  20
Thane West               20
Mumbra                   20
Goregaon East            19
Kandivali West           19
Malad East               19
Alibaug                  18
Dahisar                  18
Chembur                  18
Mulund West              18
Girgaum                  18
Fort                     18
Lower Parel              18
Borivali East            18
Matunga East             18
Ghatkopar East           18
Palghar                  18
Bandra Kurla Complex     17
Powai                    17
Khar                     17
Kandivali East           17
Bandra West              17
Worli                    17
Marol               

In [113]:
testSet.loc[(testSet.Locality == 'Andheri West'),'Locality']='Andheri'
testSet.loc[(testSet.Locality == 'Andheri East'),'Locality']='Andehri'

In [114]:
testSet.drop(testSet[testSet['Locality'] == 'Marve'].index, inplace = True)

In [115]:
testSet.drop(testSet[testSet['Locality'] == 'Nerul'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Nalasopara'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Mahim'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Mahakali'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Marve'].index, inplace = True)

In [116]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Borivali West            30
Mulund East              30
Ghatkopar West           30
Churchgate               26
Mira Road                22
Vile Parle               22
Bandra East              21
Bhandup                  20
Chandrapur Locality      20
Hinganghat Locality      20
Byculla                  20
Thane West               20
Mumbra                   20
Goregaon East            19
Kandivali West           19
Malad East               19
Alibaug                  18
Dahisar                  18
Chembur                  18
Mulund West              18
Girgaum                  18
Fort                     18
Lower Parel              18
Borivali East            18
Matunga East             18
Ghatkopar East           18
Palghar                  18
Bandra Kurla Complex     17
Powai                    17
Khar                     17
Kandivali East           17
Bandra West              17
Worli                    17
Marol               

In [117]:
testSet.loc[(testSet.Locality == 'Andehri West'),'Locality']='Andheri'
testSet.loc[(testSet.Locality == 'Andehri East'),'Locality']='Andehri'

In [118]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Mulund East              30
Ghatkopar West           30
Borivali West            30
Churchgate               26
Vile Parle               22
Mira Road                22
Bandra East              21
Thane West               20
Bhandup                  20
Hinganghat Locality      20
Chandrapur Locality      20
Byculla                  20
Mumbra                   20
Malad East               19
Kandivali West           19
Goregaon East            19
Fort                     18
Chembur                  18
Alibaug                  18
Lower Parel              18
Palghar                  18
Girgaum                  18
Ghatkopar East           18
Mulund West              18
Dahisar                  18
Matunga East             18
Borivali East            18
Khar                     17
Bandra West              17
Bandra Kurla Complex     17
Marol                    17
Worli                    17
Kandivali East           17
Powai               

In [119]:
testSet.loc[(testSet.Locality == 'Andehri'),'Locality']='Andheri'

In [120]:
testSet['Locality'].value_counts()

Juhu                     33
Malad West               31
Mulund East              30
Ghatkopar West           30
Borivali West            30
Churchgate               26
Andheri                  23
Vile Parle               22
Mira Road                22
Bandra East              21
Thane West               20
Bhandup                  20
Hinganghat Locality      20
Chandrapur Locality      20
Byculla                  20
Mumbra                   20
Malad East               19
Goregaon East            19
Kandivali West           19
Matunga East             18
Palghar                  18
Girgaum                  18
Mulund West              18
Chembur                  18
Borivali East            18
Alibaug                  18
Ghatkopar East           18
Lower Parel              18
Dahisar                  18
Fort                     18
Marol                    17
Powai                    17
Kandivali East           17
Bandra Kurla Complex     17
Worli                    17
Bandra West         

In [122]:
testSet.loc[(testSet.Locality == 'Kamala Mills Compound'),'Locality']='Lower Parel'
testSet.loc[(testSet.Locality == 'Mohammad Ali Road'),'Locality']='Andheri'
testSet.loc[(testSet.Locality == 'Mumbai Central'),'Locality']='Grant Road'
testSet.drop(testSet[testSet['Locality'] == 'New Panvel'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Airoli'].index, inplace = True)

In [126]:
testSet.drop(testSet[testSet['Locality'] == 'Hinganghat Locality'].index, inplace = True)
testSet.drop(testSet[testSet['Locality'] == 'Chandrapur Locality'].index, inplace = True)

In [129]:
testSet.reset_index(drop=True, inplace = True)
testSet['Locality'].value_counts()

Juhu                    33
Malad West              31
Mulund East             30
Borivali West           30
Ghatkopar West          30
Andheri                 28
Churchgate              26
Mira Road               22
Vile Parle              22
Lower Parel             21
Bandra East             21
Bhandup                 20
Byculla                 20
Thane West              20
Mumbra                  20
Goregaon East           19
Kandivali West          19
Malad East              19
Chembur                 18
Alibaug                 18
Mulund West             18
Girgaum                 18
Fort                    18
Dahisar                 18
Borivali East           18
Ghatkopar East          18
Palghar                 18
Matunga East            18
Marol                   17
Kandivali East          17
Powai                   17
Bandra Kurla Complex    17
Khar                    17
Worli                   17
Bandra West             17
Vikhroli                16
Jogeshwari              15
K

In [146]:
MumbaiMap = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, label in zip(testSet['Latitude'], testSet['Longitude'], testSet['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(MumbaiMap)
MumbaiMap

Given Below is just the initiall analysis in the tabular form and these results are later compared in the actual report.

In [148]:
mumbaiGrouped = testSet.groupby('Locality').mean().reset_index()
mumbaiGrouped

,Locality,Average Cost for Two,American,Asian,Awadhi,BBQ,Bakery,Bar Food,Belgian,Beverages,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
0,Alibaug,455.555556,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.166667,...,0.000000,0.000000,0.000000,0.111111,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000
1,Andheri,492.857143,0.000000,0.035714,0.000000,0.071429,0.071429,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000
2,Bandra East,438.095238,0.000000,0.047619,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000
3,Bandra Kurla Complex,1458.823529,0.058824,0.117647,0.000000,0.000000,0.058824,0.000000,0.0,0.058824,...,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bandra West,944.117647,0.058824,0.176471,0.000000,0.000000,0.294118,0.000000,0.0,0.000000,...,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000
5,Bhandup,345.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,...,0.100000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
6,Borivali East,377.777778,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.0,0.055556,...,0.055556,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Borivali West,461.666667,0.000000,0.033333,0.000000,0.000000,0.133333,0.000000,0.0,0.300000,...,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333
8,Byculla,422.500000,0.000000,0.000000,0.000000,0.000000,0.100000,0.050000,0.0,0.000000,...,0.200000,0.000000,0.000000,0.150000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000
9,Chakala,385.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [150]:
num_top_venues = 5

for cuis in mumbaiGrouped['Locality']:
    print("----"+cuis+"----")
    temp = mumbaiGrouped[mumbaiGrouped['Locality'] == cuis].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alibaug----
          venue  freq
0     Fast Food  0.39
1       Malwani  0.22
2  North Indian  0.22
3     Beverages  0.17
4       Seafood  0.17


----Andheri----
          venue  freq
0  North Indian  0.46
1       Chinese  0.43
2     Fast Food  0.25
3       Mughlai  0.21
4       Seafood  0.11


----Bandra East----
          venue  freq
0       Chinese  0.43
1  North Indian  0.38
2        Bakery  0.14
3       Mughlai  0.14
4     Fast Food  0.14


----Bandra Kurla Complex----
          venue  freq
0     Fast Food  0.29
1       Chinese  0.29
2  North Indian  0.24
3         Asian  0.12
4       Italian  0.12


----Bandra West----
      venue  freq
0    Bakery  0.29
1  Desserts  0.24
2   Chinese  0.18
3     Asian  0.18
4   Italian  0.12


----Bhandup----
          venue  freq
0       Chinese  0.65
1  North Indian  0.25
2   Street Food  0.15
3  South Indian  0.10
4        Mithai  0.10


----Borivali East----
          venue  freq
0     Fast Food  0.44
1        Bakery  0.22
2  North Indian

In [162]:
temp = mumbaiGrouped[mumbaiGrouped['Locality'] == 'Alibaug'].T.reset_index()
temp.columns = ['Alibaug','freq']
temp = temp.iloc[2:]
temp['freq'] = temp['freq'].astype(float)
testTemp = temp.sort_values('freq',ascending = False).reset_index(drop=True).head(5)

In [163]:
testTemp.T

,0,1,2,3,4
Alibaug,Fast Food,Malwani,North Indian,Beverages,Seafood
freq,0.388889,0.222222,0.222222,0.166667,0.166667


In [166]:
testTemp.T.drop(['freq'])

,0,1,2,3,4
Alibaug,Fast Food,Malwani,North Indian,Beverages,Seafood


In [237]:
topCuisFrame = pd.DataFrame()

In [238]:
def getFrame(cuisFrame):
    for cuis in mumbaiGrouped['Locality']:
        temp = mumbaiGrouped[mumbaiGrouped['Locality'] == cuis].T.reset_index()
        temp.columns = [cuis,'freq']
        temp = temp.iloc[2:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        testTemp = temp.sort_values('freq',ascending = False).reset_index(drop=True).head(5)
        testTemp = testTemp.T.drop(['freq'])
        cuisFrame = pd.concat([cuisFrame,testTemp])
        
    return cuisFrame

In [239]:
topCuisFrame = getFrame(topCuisFrame)

In [240]:
topCuisFrame.columns=['1st','2nd','3rd','4th','5th']
topCuisFrame.head()

,1st,2nd,3rd,4th,5th
Alibaug,Fast Food,Malwani,North Indian,Beverages,Seafood
Andheri,North Indian,Chinese,Fast Food,Mughlai,Seafood
Bandra East,Chinese,North Indian,Bakery,Mughlai,Fast Food
Bandra Kurla Complex,Fast Food,Chinese,North Indian,Asian,Italian
Bandra West,Bakery,Desserts,Chinese,Asian,Italian


In [241]:
export_csv = topCuisFrame.to_csv (r'C:\Users\prono\Desktop\topCuisFrameNoFreq.csv', index = True, header=True)

In [229]:
export_csv = mumbaiGrouped.to_csv (r'C:\Users\prono\Desktop\mumbaiGrouped.csv', index = None, header=True)

In [231]:
testSet.describe()

,Average Cost for Two,American,Asian,Awadhi,BBQ,Bakery,Bar Food,Belgian,Beverages,Biryani,...,South Indian,Spanish,Sri Lankan,Street Food,Sushi,Tea,Thai,Turkish,Vietnamese,Wraps
count,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.0,871.000000,871.000000,...,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000,871.000000
mean,503.605052,0.014925,0.021814,0.001148,0.003444,0.092997,0.002296,0.0,0.096441,0.052813,...,0.114811,0.001148,0.001148,0.090700,0.002296,0.005741,0.011481,0.001148,0.001148,0.004592
std,461.057356,0.121324,0.146160,0.033884,0.058621,0.290594,0.047891,0.0,0.295365,0.223788,...,0.318976,0.033884,0.033884,0.287348,0.047891,0.075592,0.106594,0.033884,0.033884,0.067650
min,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,550.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [232]:
export_csv = testSet.to_csv (r'C:\Users\prono\Desktop\testSet.csv', index = None, header=True)

#### At this point I have gotten enough dta to perform an analysis. The 'topCuisFrame' gives me an idea of what the most common cuisines in restaurants in the particular locality are, the 'mumbaiGrouped' frame allows me to see what the localities can cost around. The final analysis of all this data is added to the reoport. The data visualization is being done with the help of excel.

If you have any enquiries about this code, feel free to email me on pronoy.55@gmail.com